<a href="https://colab.research.google.com/github/serahtouati/TSP_Projet/blob/main/Notebook/TSP_Projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Génération des villes et calcul des distances


## 1. Importation du fichier csv contenant les villes

In [ ]:
import pandas as pd

# Charger les villes depuis GitHub
url = "https://github.com/serahtouati/TSP_Projet/raw/main/data/fr.csv"
cities = pd.read_csv(url)
cities.head()


,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Paris,48.8567,2.3522,France,FR,Île-de-France,primary,11060000,2148271
1,Bordeaux,44.8400,-0.5800,France,FR,Nouvelle-Aquitaine,admin,994920,994920
2,Marseille,43.2964,5.3700,France,FR,Provence-Alpes-Côte d’Azur,admin,873076,873076
3,Lyon,45.7600,4.8400,France,FR,Auvergne-Rhône-Alpes,admin,522250,522250
4,Toulouse,43.6045,1.4440,France,FR,Occitanie,admin,504078,504078


##2. Création des listes de villes

In [ ]:
def load_cities_from_csv(url, n=20):
    # Charger le fichier CSV depuis le lien
    cities_csv = pd.read_csv(url)

    # Extraire les colonnes nécessaires (nom, latitude, longitude)
    cities = cities_csv[['city', 'lat', 'lng']].head(n).values.tolist()

    # Transformer en une liste de tuples (nom, lat, lng)
    cities = [(name, lat, lng) for name, lat, lng in cities]

    return cities

# Utilisation
cities_list = load_cities_from_csv(url, n=20)
print(cities_list)


[('Paris', 48.8567, 2.3522), ('Bordeaux', 44.84, -0.58), ('Marseille', 43.2964, 5.37), ('Lyon', 45.76, 4.84), ('Toulouse', 43.6045, 1.444), ('Nice', 43.7034, 7.2663), ('Nantes', 47.2181, -1.5528), ('Montpellier', 43.6119, 3.8772), ('Strasbourg', 48.5833, 7.7458), ('Lille', 50.6278, 3.0583), ('Rennes', 48.1147, -1.6794), ('Toulon', 43.1258, 5.9306), ('Reims', 49.2628, 4.0347), ('Saint-Étienne', 45.4347, 4.3903), ('Le Havre', 49.49, 0.1), ('Dijon', 47.3167, 5.0167), ('Grenoble', 45.1715, 5.7224), ('Angers', 47.4736, -0.5542), ('Villeurbanne', 45.7667, 4.8803), ('Nîmes', 43.8383, 4.3597)]


##3. Calcul des distances entre les villes

In [ ]:
import math

def calculate_distance_matrix(cities):
    n = len(cities)
    matrix = [[0.0] * n for _ in range(n)]

    for i in range(n):
        for j in range(n):
            if i != j:
                # Coordonnées des villes
                lat1, lon1 = cities[i][1], cities[i][2]
                lat2, lon2 = cities[j][1], cities[j][2]

                # Calcul de la distance avec la formule de Haversine
                rayon = 6371  # Rayon de la Terre en km
                dlat = math.radians(lat2 - lat1)
                dlon = math.radians(lon2 - lon1)
                a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
                c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
                distance = rayon * c
                matrix[i][j] = distance

    return matrix

# Utilisation
distance_matrix = calculate_distance_matrix(cities_list)

# Afficher un extrait de la matrice
import numpy as np
np.set_printoptions(precision=2)
np.array(distance_matrix)[:5, :5]  # Les 5 premières lignes et colonnes


array([[  0.  , 499.11, 660.5 , 392.06, 588.17],
       [499.11,   0.  , 505.26, 435.99, 211.85],
       [660.5 , 505.26,   0.  , 277.14, 318.74],
       [392.06, 435.99, 277.14,   0.  , 359.85],
       [588.17, 211.85, 318.74, 359.85,   0.  ]])

##4. Heuristique du Plus Proche Voisin

In [ ]:
def nearest_neighbor(distance_matrix):

    n = len(distance_matrix)
    visited = [False] * n
    path = [0]
    visited[0] = True
    current_city = 0

    for _ in range(n - 1):
        nearest = None
        min_dist = float('inf')
        for city in range(n):
            if not visited[city] and distance_matrix[current_city][city] < min_dist:
                nearest = city
                min_dist = distance_matrix[current_city][city]
        path.append(nearest)
        visited[nearest] = True
        current_city = nearest

    path.append(0)
    return path

# Test
path_nn = nearest_neighbor(distance_matrix)
print("Chemin trouvé :", path_nn)

# Calcul de la distance totale
def calculate_total_distance(path, distance_matrix):
    return sum(
        distance_matrix[path[i]][path[i + 1]] for i in range(len(path) - 1)
    )

total_distance_nn = calculate_total_distance(path_nn, distance_matrix)
print(f"Distance totale parcourue (Nearest Neighbor) : {total_distance_nn:.2f} km")


Chemin trouvé : [0, 12, 9, 14, 10, 6, 17, 1, 4, 7, 19, 2, 11, 5, 16, 18, 3, 13, 15, 8, 0]
Distance totale parcourue (Nearest Neighbor) : 3157.88 km
